<a href="https://colab.research.google.com/github/N1ckg4m3s/assistente_virtual_simples/blob/main/Assistente_ViniTabacaria_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Caminho_Modelo = '/content/ChatBotModels'

Caminho_DataSet = '/content/DataSetParts/'

Caminho_Arquivos_DataSet=[]

# **TODAS AS IMPORTAÇÕES E CARREGAMENTOS**

Importando bibliotecas

In [ ]:
import os
from google.colab import drive

import shutil
import os

import gc
import json


from transformers import (GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TrainerCallback,
                          get_linear_schedule_with_warmup, GPT2Config)
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.optim import AdamW

import nltk
from nltk.translate.bleu_score import sentence_bleu

nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Importando DataSet

In [ ]:
## baixar o Dataset
!git clone https://github.com/N1ckg4m3s/Chat_bot_ViniTabacaria.git

Cloning into 'Chat_bot_ViniTabacaria'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 27 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 60.11 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (3/3), done.


Importando dos pesos do modelo

In [ ]:
drive.mount('/content/drive')

!ln -s /content/drive/MyDrive/ChatBotModels /content/ChatBotModels

Mounted at /content/drive


# **FUNÇÕES E CLASSES AUXILIARES**

In [ ]:
def tokenize_data(inputs, outputs, tokenizer, max_length=1024):
    # Tokenizando as entradas e saídas, com max_length garantido para ambos
    input_encodings = tokenizer(inputs, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    output_encodings = tokenizer(outputs, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

    # Aqui, truncamos ou ajustamos os labels para ter o mesmo comprimento que as entradas
    output_encodings['labels'] = output_encodings['input_ids'].clone()

    # Ajustar os labels para que o comprimento seja igual ao das entradas
    output_encodings['labels'] = output_encodings['labels'][:, :input_encodings['input_ids'].size(1)]

    return input_encodings, output_encodings


In [ ]:
def compute_metrics(eval_pred):
    logits_here, labels_here = eval_pred

    # Garantir que logits e labels sejam tensores
    logits_here = torch.tensor(logits_here) if not isinstance(logits_here, torch.Tensor) else logits_here
    labels_here = torch.tensor(labels_here) if not isinstance(labels_here, torch.Tensor) else labels_here

    # Calcula a perda
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(logits_here.view(-1, logits_here.size(-1)), labels_here.view(-1))
    perplexity = torch.exp(loss)

    # Transformar logits em previsões de classe
    preds = torch.argmax(logits_here, dim=-1)
    preds_flat = preds.view(-1).cpu().numpy()
    labels_flat = labels_here.view(-1).cpu().numpy()

    # Excluir valores ignorados (-100)
    valid_indices = labels_flat != -100
    preds_flat = preds_flat[valid_indices]
    labels_flat = labels_flat[valid_indices]

    # Métricas de precisão
    precision = precision_score(labels_flat, preds_flat, average='macro', zero_division=0)

    # Criar uma métrica combinada (exemplo: média ponderada)
    combined_metric = (1000 * precision) + (1000 / (1 + perplexity.item()))

    return {
        "loss": loss.item(),
        "perplexity": perplexity.item(),
        "precision": precision,
        "combined_metric": combined_metric
    }

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, input_encodings, output_encodings):
        # Verifique se o número de exemplos é o mesmo
        assert len(input_encodings['input_ids']) == len(output_encodings['labels']), \
            f"Mismatch in input and output sizes: {len(input_encodings['input_ids'])} != {len(output_encodings['labels'])}"

        self.input_ids = input_encodings['input_ids']
        self.attention_mask = input_encodings['attention_mask']
        self.labels = output_encodings['labels']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

In [ ]:
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3):
        self.patience = patience
        self.best_metric = float('-inf')
        self.stopped_epoch = 0

    def on_epoch_end(self, args, state, control, **kwargs):
        # A métrica já estará armazenada em state.best_metric
        combined_metric = state.best_metric
        if combined_metric > self.best_metric:
            self.best_metric = combined_metric
            self.stopped_epoch = 0  # Reset se houver melhoria
        else:
            self.stopped_epoch += 1

        if self.stopped_epoch >= self.patience:
            print(f"Early stopping triggered at epoch {state.epoch}")
            control.should_early_stop = True
        return control

# **CARREGANDO O TOKENIZADOR**

In [ ]:
try:
    tokenizer = GPT2Tokenizer.from_pretrained(Caminho_Modelo)
except:
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

# **CARREGANDO O DATASET**

Carregando dados do DataSet


In [ ]:
Caminho_Arquivos_DataSet=[]
# Carregar o arquivo JSON
with open("/content/Chat_bot_ViniTabacaria/DataSet/modelos_gerados.json", "r") as file:
    data = json.load(file)

if not os.path.exists(Caminho_DataSet):
    os.makedirs(Caminho_DataSet)

# Dividir os dados em 10 partes
num_files = 10
chunk_size = len(data) // num_files
chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

# Se houver sobra de elementos (quando len(data) não for múltiplo de 10), adicionar os extras ao último arquivo
if len(data) % num_files != 0:
    chunks[-1].extend(data[num_files * chunk_size:])

# Salvar os dados divididos em 10 arquivos
for i, chunk in enumerate(chunks):
    output_file = f"{Caminho_DataSet}part_{i+1}.json"  # Caminho correto do arquivo
    with open(output_file, "w") as file:
        json.dump(chunk, file, indent=4)

    print(f"Arquivo {output_file} salvo com sucesso.")

    Caminho_Arquivos_DataSet.append(output_file)

# Excluir as variáveis para liberar memória
del data
del chunks

# Forçar a coleta de lixo para liberar memória
gc.collect()

print("Arquivos divididos, salvos e memória liberada com sucesso.")


Arquivo /content/DataSetParts/part_1.json salvo com sucesso.
Arquivo /content/DataSetParts/part_2.json salvo com sucesso.
Arquivo /content/DataSetParts/part_3.json salvo com sucesso.
Arquivo /content/DataSetParts/part_4.json salvo com sucesso.
Arquivo /content/DataSetParts/part_5.json salvo com sucesso.
Arquivo /content/DataSetParts/part_6.json salvo com sucesso.
Arquivo /content/DataSetParts/part_7.json salvo com sucesso.
Arquivo /content/DataSetParts/part_8.json salvo com sucesso.
Arquivo /content/DataSetParts/part_9.json salvo com sucesso.
Arquivo /content/DataSetParts/part_10.json salvo com sucesso.
Arquivo /content/DataSetParts/part_11.json salvo com sucesso.
Arquivos divididos, salvos e memória liberada com sucesso.


Separando parte para Teste e Validação

In [ ]:
def GetTrain_Val_Data(Caminho_Arquivos_DataSet, tokenizer):
    # Carregar os dados do arquivo
    with open(Caminho_Arquivos_DataSet, "r") as file:
        data = json.load(file)

    # Separar dados de treino e validação
    inputs_train, inputs_val, outputs_train, outputs_val = train_test_split(
        [item["input"] for item in data],
        [item["output"] for item in data],
        test_size=0.2,
        shuffle=True,
        random_state=85
    )

    # Criar datasets de treino e validação
    input_encode_train, output_encode_train = tokenize_data(inputs_train, outputs_train, tokenizer)
    input_encode_val, output_encode_val = tokenize_data(inputs_val, outputs_val, tokenizer)

    # Criar os datasets personalizados para treino e validação
    train_dataset = CustomDataset(input_encode_train, output_encode_train)
    val_dataset = CustomDataset(input_encode_val, output_encode_val)

    # Limpar as variáveis para liberar memória
    del data, inputs_train, inputs_val, outputs_train, outputs_val, input_encode_train, output_encode_train, input_encode_val, output_encode_val

    # Forçar a coleta de lixo para liberar memória
    gc.collect()

    # Retornar os datasets
    return train_dataset, val_dataset

# **CONFIG. MODELO**


In [ ]:
num_epochs = 50

config = GPT2Config(
    n_layer=6,         # Reduz para 6 camadas
    n_embd=256,        # Tamanho do embedding reduzido
    n_head=4,          # Reduz para 4 cabeças de atenção
    n_positions=512,   # Limita o número de posições
    n_ctx=512          # Limita o contexto
)

In [ ]:
try:
    model = GPT2LMHeadModel.from_pretrained(Caminho_Modelo, config=config)
    tokenizer = GPT2Tokenizer.from_pretrained(Caminho_Modelo)
    print("Modelo e tokenizador carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o modelo e tokenizador: {e}")
    model = GPT2LMHeadModel(config)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Preparar o modelo para treinamento
model.train()

Modelo e tokenizador carregados com sucesso!


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 256)
    (wpe): Embedding(512, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=768, nx=256)
          (c_proj): Conv1D(nf=256, nx=256)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=1024, nx=256)
          (c_proj): Conv1D(nf=256, nx=1024)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=256, out_features=50257, bias=False)
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./model_output",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    fp16=True,
    max_grad_norm=1.0,
    metric_for_best_model="combined_metric",
    greater_is_better=False,
    dataloader_num_workers=2,
    report_to="none"
)

# **CARREGAR MODELO/TOKENIZADOR**

In [ ]:
device = torch.device("cpu")
model.to(device)
model.gradient_checkpointing_enable()

for Caminho in Caminho_Arquivos_DataSet:
    try:
        model = GPT2LMHeadModel.from_pretrained(Caminho_Modelo, config=config)
        tokenizer = GPT2Tokenizer.from_pretrained(Caminho_Modelo)
    except Exception as e:
        model = GPT2LMHeadModel(config)
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    tokenizer.pad_token = tokenizer.eos_token
    # Preparar o modelo para treinamento
    model.train()

    train_dataset, val_dataset = GetTrain_Val_Data(Caminho, tokenizer)

    # Criar o otimizador e o agendador de taxa de aprendizado
    optimizer = AdamW(model.parameters(), lr=5e-5)
    total_steps = len(train_dataset) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(total_steps * 0.1),
        num_training_steps=total_steps
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(patience=3)]
    )
    trainer.train()

    model.save_pretrained(Caminho_Modelo=='' and '/content/ChatBotModels' or Caminho_Modelo)
    tokenizer.save_pretrained(Caminho_Modelo=='' and '/content/ChatBotModels' or Caminho_Modelo)

Epoch,Training Loss,Validation Loss


TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
model.save_pretrained('/content/ChatBotModels')
tokenizer.save_pretrained('/content/ChatBotModels')

('/content/ChatBotModels/tokenizer_config.json',
 '/content/ChatBotModels/special_tokens_map.json',
 '/content/ChatBotModels/vocab.json',
 '/content/ChatBotModels/merges.txt',
 '/content/ChatBotModels/added_tokens.json')

# **VALIDAR MODELO**

In [ ]:
try:
    model = GPT2LMHeadModel.from_pretrained(Caminho_Modelo, config=config)
    tokenizer = GPT2Tokenizer.from_pretrained(Caminho_Modelo)
except Exception as e:
    model = GPT2LMHeadModel(config)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model.eval()

# Função para gerar resposta e calcular o loss
def gerar_resposta_com_loss(texto, max_length=60):
    # Tokenizar o texto de entrada
    inputs = tokenizer.encode(texto, return_tensors='pt')

    # Usar o mesmo texto como 'labels' para calcular a perda
    labels = inputs.clone()

    # Gerar a saída com o modelo
    outputs = model(input_ids=inputs, labels=labels)

    # A perda é armazenada no atributo 'loss'
    loss = outputs.loss
    logits = outputs.logits

    # Gerar a sequência de texto com o método generate
    outputs_generate = model.generate(
        inputs,
        max_length=max_length,            # Limitar o comprimento da resposta
        num_return_sequences=1,           # Apenas uma resposta
        num_beams=5,                      # Beam search com 5 opções
        no_repeat_ngram_size=4,           # Evitar repetições de 4-gramas
        pad_token_id=tokenizer.eos_token_id
    )

    # Decodificar a resposta gerada para texto
    resposta = tokenizer.decode(outputs_generate[0], skip_special_tokens=True)

    return resposta, loss.item()

In [ ]:

# Testar com um exemplo personalizado
texto_entrada = "Quero uma ziggy uva"
resposta_gerada, loss_gerado = gerar_resposta_com_loss(texto_entrada, max_length=100)

# Mostrar a resposta gerada e a perda calculada
print("Resposta gerada:", resposta_gerada)
print("Loss calculado:", loss_gerado)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Resposta gerada: Quero uma ziggy uvaduto],Marca=,sabor=uva)u
Loss calculado: 15.285144805908203
